In [1]:
import pandas as pd
from util.util import *
from transformers import AutoTokenizer
import random
from torch.utils.data import DataLoader
import torch
import torch.nn as nn
import torch.optim as optim
# import torch.utils.data as data
from util.node import Transformer
from torch.optim.lr_scheduler import StepLR,CosineAnnealingLR
# import math
# import copy

/home/athip/psu/.env/pytorch/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [77]:
random.randrange(1,10,1)

5

In [79]:
[1,2,3,4][0:4]

[1, 2, 3, 4]

In [2]:
tokenizer = AutoTokenizer.from_pretrained("gpt2")

In [3]:
def tokenizerL(text):
    words_with_offsets = tokenizer.backend_tokenizer.pre_tokenizer.pre_tokenize_str(text)
    new_words = [word for word, offset in words_with_offsets]
    return new_words

In [41]:
tokenizerL("qwertyuiop[]asdfghjkl;'zxcvbnm,./")

['qwertyuiop', '[]', 'asdfghjkl', ";'", 'zxcvbnm', ',./']

In [42]:
tokenizerL("1234567890-=")

['1234567890', '-=']

In [ ]:
tokenizerL(" ! @ # $ % ^ & * ( ) _ + Q W E R T Y U  I O P { } A S D F G H J K L : " " Z X C V B N M < > ? | ")

In [47]:
tokenizerL(" ")

['Ġ']

In [48]:
tokenizerL("\n")

['Ċ']

In [4]:
question = []
answer = []
all_data = []

In [5]:
data = pd.read_csv("data/WikiQACorpus/WikiQA-train.tsv", chunksize=10000,sep="\t")
for d in data:
    question += d["Question"].tolist()
    answer += d["Sentence"].tolist()
all_data = question + answer

In [ ]:
data = pd.read_csv("./data/question_and_answer_no_code01/Dataset_Python_Question_Answer.csv", chunksize=10000)
for d in data:
    question += d["Question"].tolist()
    ans = d["Answer"].map(lambda x: "\n".join(eval(x))).tolist()
    answer += ans
all_data = question + answer

In [6]:
question[:10]

['how are glacier caves formed?',
 'how are glacier caves formed?',
 'how are glacier caves formed?',
 'how are glacier caves formed?',
 'how are glacier caves formed?',
 'How are the directions of the velocity and force vectors related in a circular motion',
 'How are the directions of the velocity and force vectors related in a circular motion',
 'How are the directions of the velocity and force vectors related in a circular motion',
 'How are the directions of the velocity and force vectors related in a circular motion',
 'How are the directions of the velocity and force vectors related in a circular motion']

In [7]:
answer[:10]

['A partly submerged glacier cave on Perito Moreno Glacier .',
 'The ice facade is approximately 60 m high',
 'Ice formations in the Titlis glacier cave',
 'A glacier cave is a cave formed within the ice of a glacier .',
 'Glacier caves are often called ice caves , but this term is properly used to describe bedrock caves that contain year-round ice.',
 'In physics , circular motion is a movement of an object along the circumference of a circle or rotation along a circular path.',
 'It can be uniform, with constant angular rate of rotation (and constant speed), or non-uniform with a changing rate of rotation.',
 'The rotation around a fixed axis of a three-dimensional body involves circular motion of its parts.',
 'The equations of motion describe the movement of the center of mass of a body.',
 'Examples of circular motion include: an artificial satellite orbiting the Earth at constant height, a stone which is tied to a rope and is being swung in circles, a car turning through a curve 

In [2]:
vocab_size = 1024*2

In [3]:
bpe = BPE()

In [4]:
bpe.idx2token

<bound method BPE.idx2token of <util.util.BPE object at 0x794e8a3cbad0>>

In [5]:
bpe.load_pretrain("model/BPE_model/BPE_model_code_python01.pkl")

In [8]:
bpe.train(all_data,vocab_size=vocab_size)

count word freq


100%|██████████| 40694/40694 [00:00<00:00, 49514.93it/s] 


find alphabet


100%|██████████| 2048/2048 [01:57<00:00, 17.43it/s] 


In [12]:
bpe.save_model("model/BPE_model/BPE_model_code_python04.pkl")

In [11]:
bpe.vocab[1900:]

['Ġallow',
 'Ġforces',
 'Ġmoun',
 'ged',
 'Ġassoc',
 'Ġestim',
 'Ġdeterm',
 'Ġcame',
 'Ġturn',
 'Ġsport',
 'ĠBr',
 'aster',
 'hod',
 'Ġfield',
 'Ġimportant',
 'icro',
 'Ġ13',
 'eptember',
 'Ġgenerally',
 'Ġmovie',
 'ingu',
 'ids',
 'Ġspace',
 'ma',
 'Ġ30',
 'ait',
 'Ġfull',
 'Ġmark',
 'Ġ1990',
 'Ġanother',
 'ications',
 'These',
 'ĠAustral',
 'Ġchildren',
 'ological',
 'Ġrese',
 'ison',
 'Ġhydro',
 'ille',
 'Ġexist',
 'Ġchem',
 'Ġtechnology',
 'Ġpat',
 'born',
 'Ġfire',
 'Ġprofessional',
 'Ġfilms',
 'ulf',
 'ivision',
 'Ġmanag',
 'uck',
 '10',
 'ential',
 'sp',
 'Ġprimary',
 'Ġsignificant',
 'side',
 'ĠAt',
 'Ġsum',
 'aren',
 'Ġculture',
 'Ġtest',
 'Ġnovel',
 'Ġprevious',
 'Ġdeep',
 'Ġauthor',
 'Ġproperty',
 'Ġhold',
 'ves',
 'aper',
 'line',
 'Ġ&',
 'Ġapp',
 'ending',
 'adem',
 'Ġcurrently',
 'Ġaver',
 'Ġestablished',
 'imately',
 'ither',
 'ĠSeptember',
 'Ġtradition',
 'Ġrecorded',
 'lant',
 'Ġpar',
 'avery',
 'Ġstill',
 'Ġreceived',
 'Ġhit',
 'ĠIS',
 'Ġcauses',
 'ady',
 'Ġhand',
 'Ġ

In [9]:
len(bpe.vocab)

2048

In [33]:
decode = "".join(bpe.tokenize(answer[100])).replace("Ġ"," ").replace("Ċ","\n")

In [34]:
print(decode)

RS-68 being tested at NASA's Stennis Space Center.


In [11]:
print(answer[0])

Sure, here's the difference between a variable and an object:
**Variable:**
* A variable is a named memory location that stores a single value.
* It is a placeholder for a specific amount of data.
* Variables can hold different values throughout the program.
* They are declared using the `=` operator and assigned a value.
**Object:**
* An object is a complex data structure that contains data and methods.
* It is an instance of a class.
* Objects can have multiple variables and methods associated with them.
* They are created using the `new` keyword and contain a copy of all the variables and methods of the class.
In summary, a variable is a single piece of memory that stores a single value, while an object is a complex data structure that contains data and methods.


In [25]:
a1 = tokenizerL(answer[0])

In [23]:
tt = """Sure, here's the difference between a variable and an object:
**Variable:**
* A variable is a named memory location that stores a single value.
* It is a placeholder for a specific amount of data.
* Variables can hold different values throughout the program.
* They are declared using the `=` operator and assigned a value.
**Object:**
* An object is a complex data structure that contains data and methods.
* It is an instance of a class.
* Objects can have multiple variables and methods associated with them.
* They are created using the `new` keyword and contain a copy of all the variables and methods of the class.
In summary, a variable is a single piece of memory that stores a single value, while an object is a complex data structure that contains data and methods."""

In [26]:
a2 = tokenizerL(tt)

In [27]:
a1 == a2

True

In [6]:
train_data = dataloadercustom()

In [7]:
len(train_data)

419

In [8]:
train_dataloader = DataLoader(train_data,batch_size=64,shuffle=True)

In [9]:
qq,aa_1,aa_2 = next(iter(train_data))

In [13]:
"".join(bpe.idx2token(qq[:].cpu().tolist())).replace("Ġ"," ").replace("Ċ","\n")

'<|startoftext|> What is the difference between a variable and an object?<|endoftext|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|>'

In [15]:
print("".join(bpe.idx2token(aa_1[:].cpu().tolist())).replace("Ġ"," ").replace("Ċ","\n"))

<|startoftext|>Sure, here's the difference between a variable and an object:
**Variable:**
* A variable is a named memory location that stores a single value.
* It is a placeholder for a specific amount of data.
* Variables can hold different values throughout the program.
* They are declared using the `=` operator and assigned a value.
**Object:**
* An object is a complex data structure that contains data and methods.
* It is an instance of a class.
* Objects can have multiple variables and methods associated with them.
* They are created using the `new` keyword and contain a copy of all the variables<|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|

In [16]:
print("".join(bpe.idx2token(aa_2[:].cpu().tolist())).replace("Ġ"," ").replace("Ċ","\n"))

<|startoftext|>Sure, here's the difference between a variable and an object:
**Variable:**
* A variable is a named memory location that stores a single value.
* It is a placeholder for a specific amount of data.
* Variables can hold different values throughout the program.
* They are declared using the `=` operator and assigned a value.
**Object:**
* An object is a complex data structure that contains data and methods.
* It is an instance of a class.
* Objects can have multiple variables and methods associated with them.
* They are created using the `new` keyword and contain a copy of all the variables and<|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|

In [2]:
import pandas as pd
from util.util import *
from transformers import AutoTokenizer
import random
from torch.utils.data import DataLoader

In [2]:
train_data = dataloadercustom()
train_dataloader = DataLoader(train_data,batch_size=64,shuffle=True)

In [3]:
class WarmupCosineScheduler:
    def __init__(self, optimizer, warmup_steps, max_steps, base_lr):
        self.optimizer = optimizer
        self.warmup_steps = warmup_steps
        self.max_steps = max_steps
        self.base_lr = base_lr
        self.cosine_scheduler = CosineAnnealingLR(optimizer, T_max=(max_steps - warmup_steps))
        self.current_step = 0

    def step(self):
        if self.current_step < self.warmup_steps:
            # Linear warmup
            lr = self.base_lr * (self.current_step / self.warmup_steps)
            for param_group in self.optimizer.param_groups:
                param_group['lr'] = lr
        else:
            # Cosine annealing
            self.cosine_scheduler.step()
        
        self.current_step += 1

# # Example usage
# model = torch.nn.Linear(10, 1)  # Simple linear model
# optimizer = Adam(model.parameters(), lr=0.001)  # Base learning rate
# warmup_steps = 500
# max_steps = 5000
# scheduler = WarmupCosineScheduler(optimizer, warmup_steps, max_steps, base_lr=0.001)

# # Training loop
# for epoch in range(num_epochs):
#     for batch in dataloader:
#         # Training step
#         optimizer.zero_grad()
#         output = model(batch)
#         loss = loss_fn(output, batch)
#         loss.backward()
#         optimizer.step()
        
#         # Update learning rate
#         scheduler.step()

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
# import torch.utils.data as data
from util.node import Transformer
from torch.optim.lr_scheduler import StepLR,CosineAnnealingLR
# import math
# import copy

# data loader
train_data = dataloadercustom()
train_dataloader = DataLoader(train_data,batch_size=64,shuffle=True)

src_vocab_size = train_data.token_size
tgt_vocab_size = train_data.token_size
d_model = 128*3
num_heads = 6*1
num_layers = 6*1
d_ff = 2048//8
max_seq_length = train_data.window_size
dropout = 0.1

transformer = Transformer(src_vocab_size, tgt_vocab_size, d_model,
                          num_heads, num_layers, d_ff, max_seq_length, dropout, device=0).to(device=0)

# Generate random sample data
# (batch_size, seq_length)
src_data = torch.randint(
    1, src_vocab_size, (64//8, max_seq_length)).to(device=0)
# (batch_size, seq_length)
tgt_data = torch.randint(
    1, tgt_vocab_size, (64//8, max_seq_length)).to(device=0)

criterion = nn.CrossEntropyLoss(ignore_index=0).to(device=0)
optimizer = optim.Adam(transformer.parameters(),
                       lr=0.001, betas=(0.9, 0.95), eps=1e-9)

# Learning rate scheduler
warmup_steps = 500
max_steps = 5000
scheduler = WarmupCosineScheduler(optimizer, warmup_steps, max_steps, base_lr=0.001)
# scheduler = StepLR(optimizer, step_size=20, gamma=0.5)
# scheduler = CosineAnnealingWarmRestarts(optimizer=optimizer, )

transformer.train()

for epoch in tqdm(range(1)):
    for question, answer_in, answer_out in tqdm(train_dataloader):
        optimizer.zero_grad()
        output = transformer(question, answer_in)
        loss = criterion(output.contiguous().view(-1, tgt_vocab_size),
                         answer_out.contiguous().view(-1))
        loss.backward()
        optimizer.step()
        scheduler.step()
        print(f"Epoch: {epoch+1}, Loss: {loss.item()}")


In [13]:
from datasets import load_dataset
ds = load_dataset("microsoft/wiki_qa")

Generating train split: 100%|██████████| 20360/20360 [00:00<00:00, 948138.93 examples/s]


In [17]:
ds["train"][0]["question"]

'how are glacier caves formed?'